### read data and calc returns

In [25]:
import pandas as pd


df = pd.read_csv('../../bbgfactor/rty_2014.csv', parse_dates = True).set_index(['ID','DATE'])

In [26]:
df = df[~df.index.duplicated()].reset_index() # causes multiindex issues later

In [27]:
df = df.dropna(subset = ['close'])
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns = ['Unnamed: 0'])

df['DATE'] = pd.to_datetime(df['DATE'])
df = df.sort_values(['ID', 'DATE'])
    

In [28]:
df.head()

,ID,DATE,open,high,low,close,volume,market_cap,bid,ask,baspd
3529456,2551003D US Equity,2019-03-29,29.00,29.95,29.0000,29.95,61185.0,NaN,29.98,30.1,0.131854
3529459,2551003D US Equity,2019-04-01,31.00,31.13,30.5438,30.98,222568.0,NaN,30.98,31.0,0.239853
3529460,2551003D US Equity,2019-04-02,31.00,31.50,30.6100,31.00,40901.0,1.273198e+09,30.85,31.0,0.339595
3529461,2551003D US Equity,2019-04-03,31.00,31.00,29.9000,31.00,28411.0,1.273198e+09,30.99,31.0,0.398489
3529462,2551003D US Equity,2019-04-04,31.14,31.23,30.9700,31.00,369391.0,1.273198e+09,30.99,31.0,0.196222


In [29]:
# read SPX data

In [39]:
spxdf = pd.read_csv('../../bbgfactor/idx.csv', parse_dates = True).set_index(['ID','DATE'])

In [40]:
spxdf = spxdf[~spxdf.index.duplicated()].reset_index() # causes multiindex issues later

In [41]:
spxdf = spxdf.dropna(subset = ['close'])

if 'Unnamed: 0' in spxdf.columns:
    spxdf = spxdf.drop(columns = ['Unnamed: 0'])

spxdf['DATE'] = pd.to_datetime(spxdf['DATE'])
spxdf = spxdf.sort_values('DATE')

In [42]:
spxdf = spxdf.query('ID == "SPX Index"')

### functions - ret calc and makeready

In [43]:
days = 63

In [44]:
spxdf

,ID,DATE,open,high,low,close,volume
0,SPX Index,2004-12-31,1213.55,1217.33,1211.65,1211.92,6.754980e+08
3,SPX Index,2005-01-03,1211.92,1217.90,1200.30,1202.08,1.331432e+09
4,SPX Index,2005-01-04,1202.08,1205.84,1185.39,1188.05,1.549158e+09
5,SPX Index,2005-01-05,1188.05,1192.75,1183.72,1183.74,1.425108e+09
6,SPX Index,2005-01-06,1183.74,1191.63,1183.23,1187.89,1.323134e+09
...,...,...,...,...,...,...,...
65787,SPX Index,2025-02-03,5969.65,6022.13,5923.93,5994.57,8.812059e+08
65788,SPX Index,2025-02-04,5998.14,6042.48,5990.87,6037.88,8.242833e+08
65789,SPX Index,2025-02-05,6020.45,6062.86,6007.06,6061.48,8.615793e+08
65790,SPX Index,2025-02-06,6072.22,6084.03,6046.83,6083.57,8.158219e+08


In [45]:
# df[['ID','DATE']].duplicated().any()

spxdf[['ID','DATE']].duplicated().any()

np.False_

In [46]:
df = df.merge(spxdf[['DATE','close']].rename(columns ={'close':'spx_close'}), on = 'DATE', how = 'left')

In [47]:
df['forward_return'] = (df.groupby('ID')['close']
                          .shift(-days) / df['close'] - 1)

df['spx_forward_return'] = (df.groupby('ID')['spx_close']
                          .shift(-days) / df['spx_close'] - 1)

df['relative_return'] = (
        df['forward_return'] - df['spx_forward_return']
    )

In [48]:
def calculate_returns(df):
    """Helper function to calculate returns and excess returns"""
    df = df.copy()
    df['stock_return'] = df.groupby('ID')['close'].pct_change()
    df['spy_return'] = df.groupby('ID')['spx_close'].pct_change()
    df['stock_excess_return'] = df['stock_return'] - df['spy_return']

    return df

### functions - rolling reg / var

In [49]:

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from joblib import Parallel, delayed

def rolling_residual_variance(df, window_size, dependent_var, independent_vars):
    """
    Performs rolling regression in parallel using joblib.

    Args:
        df: Pandas DataFrame containing the data.
        window_size: Size of the rolling window.
        dependent_var: Name of the dependent variable column.
        independent_vars: List of names of independent variable columns.

    Returns:
        Pandas DataFrame with the regression coefficients for each window.
        Returns None if there are issues.
    """

    n_rows = len(df)
    results = []

    def _regress_window(i):
        if i < window_size -1:
            return None # Handle edge cases at beginning of dataframe
        window_data = df.iloc[i - window_size + 1:i + 1]

        X = window_data[independent_vars].values
        y = window_data[dependent_var].values.reshape(-1,1) # Reshape y for sklearn

        if len(window_data) < window_size or np.any(np.isnan(X)) or np.any(np.isnan(y)):
            return None  # Handle cases where the window is incomplete or contains NaNs.
        
        model = LinearRegression()
        model.fit(X, y)
        y_pred = model.predict(X)

        # Step 3: Calculate the residuals
        residuals = y - y_pred

        # Step 4: Compute the residual variance
        residual_variance = np.var(residuals, ddof=1) 

        return {'index': df.index[i], 'residual_variance': residual_variance} # Include index for proper merging


    results = Parallel(n_jobs=-1)(delayed(_regress_window)(i) for i in range(n_rows))
    
    

    # Filter out None results (from edge cases or NaN windows)
    valid_results = [r for r in results if r is not None]

    if not valid_results: # Check if all results are invalid
        return None

    results_df = pd.DataFrame(valid_results)#.set_index('index')
    
    # Handle multiindex
    if isinstance(results_df['index'].iloc[0], tuple):
        results_df[list(df.index.names)] = results_df['index'].apply(pd.Series)
        results_df = results_df.drop(columns='index').set_index(list(df.index.names))
    else:
        results_df = results_df.set_index('index')
        
    return results_df

def rolling_regression(df, window_size, dependent_var, independent_vars, reg_type='OLS', alpha=1.0):
    """
    Performs rolling regression in parallel using joblib, with options for OLS, Ridge, and Lasso.
    Returns np.nan for coefficients when X or y contains NaN values.

    Args:
        df: Pandas DataFrame containing the data.
        window_size: Size of the rolling window.
        dependent_var: Name of the dependent variable column.
        independent_vars: List of names of independent variable columns.
        reg_type: Type of regression to perform ('OLS', 'Ridge', 'Lasso'). Default is 'OLS'.
        alpha: Regularization strength for Ridge and Lasso. Default is 1.0.

    Returns:
        Pandas DataFrame with the regression coefficients for each window, indexed by the original DataFrame's index.
        Returns None if there are issues.
    """

    n_rows = len(df)
    results = []

    def _regress_window(i):
        if i < window_size - 1:
            return {'index': df.index[i],
                   'intercept': np.nan,
                   **dict(zip(independent_vars, [np.nan] * len(independent_vars)))}

        window_data = df.iloc[i - window_size + 1:i + 1]

        X = window_data[independent_vars].values
        y = window_data[dependent_var].values.reshape(-1,1)

        # Return NaN coefficients if window contains NaN or is incomplete
        if len(window_data) < window_size or np.any(np.isnan(X)) or np.any(np.isnan(y)):
            return {'index': df.index[i],
                   'intercept': np.nan,
                   **dict(zip(independent_vars, [np.nan] * len(independent_vars)))}

        if reg_type.upper() == 'OLS':
            model = LinearRegression()
        elif reg_type.upper() == 'RIDGE':
            model = Ridge(alpha=alpha)
        elif reg_type.upper() == 'LASSO':
            model = Lasso(alpha=alpha)
        else:
            raise ValueError("Invalid reg_type. Choose 'OLS', 'Ridge', or 'Lasso'.")

        model.fit(X, y)
        coefs = model.coef_.flatten()
        intercept = model.intercept_

        return {'index': df.index[i], 'intercept': intercept, **dict(zip(independent_vars, coefs))}

    results = Parallel(n_jobs=-1)(delayed(_regress_window)(i) for i in range(n_rows))

    # All results should be valid now since we're returning NaN instead of None
    results_df = pd.DataFrame(results)

    # Handle multiindex
    if isinstance(results_df['index'].iloc[0], tuple):
        results_df[list(df.index.names)] = results_df['index'].apply(pd.Series)
        results_df = results_df.drop(columns='index').set_index(list(df.index.names))
    else:
        results_df = results_df.set_index('index')

    return results_df



### calculate metrics 

In [50]:
mdf = calculate_returns(df).set_index(['ID','DATE'])

/var/folders/sw/kl2bvz9d5275zz0p278kz9nc0000gn/T/ipykernel_96661/207521062.py:5: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['spy_return'] = df.groupby('ID')['spx_close'].pct_change()


### technical factors

In [51]:
beta_period = 252
vol_period = 63

In [ ]:
mdf['beta'] = mdf\
.groupby('ID', group_keys = False).apply(lambda x: rolling_regression(x, window_size =  beta_period, dependent_var = 'stock_return', independent_vars = ['spy_return']))\
.drop(columns = 'intercept')\
.rename(columns = {'spy_return':'beta'})



In [52]:


mdf['volatility'] = mdf.groupby('ID',group_keys = False)['stock_return'].rolling(vol_period).std().mul(np.sqrt(252)).reset_index(level = 0, drop = True).rename('volatility')



In [53]:

mdf['avg_volm_to_cap'] = mdf.groupby('ID', group_keys = False).apply(lambda x: x['volume'].rolling(vol_period).mean()/(x['market_cap']/1000000)).rename('avg_volm_to_cap')


In [ ]:


mdf['volume_trend'] = mdf.groupby('ID', group_keys = False).apply(lambda x: rolling_regression(x.assign(trend = lambda x:np.arange(len(x))), window_size = vol_period , dependent_var = 'volume', independent_vars = ['trend'])).rename(columns = {'trend':'volume_trend'}).drop(columns = 'intercept')


In [ ]:



mdf['residual_variance'] = mdf.groupby('ID', group_keys = False).apply(lambda x: rolling_residual_variance(x, window_size = vol_period , dependent_var = 'stock_return', independent_vars = ['spy_return']))



In [ ]:
def compound_returns(returns):
    return (1 + returns).prod() - 1

# List of rolling periods to calculate
periods = [1, 2, 3, 6, 12]*21

# Calculate rolling compounded returns for each period
for period in periods:
    # Stock returns
    mdf[f'stock_return_{period}m'] = mdf.groupby('ID')['stock_return'].rolling(
        window=period, min_periods=period
    ).apply(compound_returns).reset_index(level = 0, drop = True).values

    # SPY returns
    mdf[f'spy_return_{period}m'] = mdf.groupby('ID')['spy_return'].rolling(
        window=period, min_periods=period
    ).apply(compound_returns).reset_index(level = 0, drop = True).values

    # Calculate excess returns (stock - spy)
    mdf[f'rs_{period}m'] = mdf[f'stock_return_{period}m'] - mdf[f'spy_return_{period}m']

In [ ]:
mdf['3mrs_3mago'] = mdf.groupby('ID')['rs_3m'].shift(3)
mdf['3mrs_6mago'] = mdf.groupby('ID')['rs_3m'].shift(6)
mdf['3mrs_9mago'] = mdf.groupby('ID')['rs_3m'].shift(9)

### value factors

In [ ]:
# earnings to price

In [ ]:
mdf['eps_to_price'] = mdf.groupby('ID').apply(lambda x: x['eps']/x['px_last_splits']).reset_index(0, drop = True)


mdf['eps_to_price_trend']= mdf.groupby('ID', group_keys = False)\
.apply(lambda x: rolling_regression(x.assign(trend = lambda x:np.arange(len(x))), window_size = 24 , dependent_var = 'eps_to_price', independent_vars = ['trend']))\
.rename(columns = {'trend':'eps_to_price_trend'}).drop(columns = 'intercept')


In [ ]:
# sales to price 

In [ ]:
mdf['sales_to_price'] = mdf.groupby('ID').apply(lambda x: x['sales']/x['px_last_splits']).reset_index(0, drop = True)


mdf['sales_to_price_trend']= mdf.groupby('ID', group_keys = False)\
.apply(lambda x: rolling_regression(x.assign(trend = lambda x:np.arange(len(x))), window_size = 24 , dependent_var = 'sales_to_price', independent_vars = ['trend']))\
.rename(columns = {'trend':'sales_to_price_trend'}).drop(columns = 'intercept')


In [ ]:
# cash to price

In [ ]:
mdf['fcf_calc'] = mdf['cfo_ltm_a'] + mdf['capex'] + mdf['dvd'] 

mdf['cash_to_price'] = mdf['fcf_calc'] / mdf['px_last_splits']

mdf['cash_to_price_trend']= mdf.groupby('ID', group_keys = False)\
.apply(lambda x: rolling_regression(x.assign(trend = lambda x:np.arange(len(x))), window_size = 24 , dependent_var = 'cash_to_price', independent_vars = ['trend']))\
.rename(columns = {'trend':'cash_to_price_trend'}).drop(columns = 'intercept')


In [ ]:
# dividend to price 

In [ ]:
mdf['div_to_price'] = np.abs(mdf['dvd']) / mdf['px_last_splits']

mdf['div_to_price_trend']= mdf.groupby('ID', group_keys = False)\
.apply(lambda x: rolling_regression(x.assign(trend = lambda x:np.arange(len(x))), window_size = 24 , dependent_var = 'div_to_price', independent_vars = ['trend']))\
.rename(columns = {'trend':'div_to_price_trend'}).drop(columns = 'intercept')


In [ ]:
# book to price


In [ ]:
mdf['book_to_price'] = mdf['book_value'] / mdf['cur_mkt_cap']

mdf['book_to_price_trend']= mdf.groupby('ID', group_keys = False)\
.apply(lambda x: rolling_regression(x.assign(trend = lambda x:np.arange(len(x))), window_size = 24 , dependent_var = 'book_to_price', independent_vars = ['trend']))\
.rename(columns = {'trend':'book_to_price_trend'}).drop(columns = 'intercept')


In [ ]:
mdf.columns.tolist()

In [54]:
fdf = mdf[['forward_return',
           'stock_excess_return',
           'market_cap',
           'avg_volm_to_cap',
             'volume',
             'volatility',
             ]]
           

In [ ]:
fdf = mdf[['stock_return',
           'cur_mkt_cap',
           'px_last_splits',
           'beta',
             'volatility',
             'avg_volm_to_cap',
             'volume_trend',
             'residual_variance',
             'stock_return_1m',
             'rs_1m',
             'stock_return_2m',

             'rs_2m',
             'stock_return_3m',

             'rs_3m',
             'stock_return_6m',

             'rs_6m',
             'stock_return_12m',

             'rs_12m',
             '3mrs_3mago',
             '3mrs_6mago',
             '3mrs_9mago',
             'eps_to_price',
             'eps_to_price_trend',
             'sales_to_price',
             'sales_to_price_trend',
             'fcf_calc',
             'cash_to_price',
             'cash_to_price_trend',
             'div_to_price',
             'div_to_price_trend',
             'book_to_price',
             'book_to_price_trend']]
           

In [56]:
fcols = fdf.columns.to_list()
fcols.remove('forward_return')

In [57]:
fdf = fdf.groupby('DATE').transform(lambda x: x.fillna(x.mean()))

In [58]:
from scipy.stats.mstats import winsorize
from scipy.stats import zscore

# df[numcols] = df[numcols].groupby('ITERATION_DATE').transform(lambda x: winsorize(x, limits = (0.01,0.01)))
fdf[fcols] = fdf[fcols].groupby('DATE').transform(lambda x: zscore(x).clip(-3,3))

In [59]:

# now we are calculating forward returns, so if standing on Dec 1, its 3m forward returns and Dec 1 features, so we are good, nothign to shift 
fdf = fdf.dropna(subset = ['forward_return'])
# fdf[fcols] = fdf.groupby(level = 'ID')[fcols].shift(days)

In [60]:
fdf.xs('2018-12-31', level = 'DATE').head()

,forward_return,stock_excess_return,market_cap,avg_volm_to_cap,volume,volatility
ID,,,,,,
AAMI UN Equity,0.284644,0.194273,-0.030936,-0.106074,0.309433,-0.418957
AAOI UQ Equity,-0.213869,0.523236,-0.715685,1.000699,0.408269,0.613026
AAT UN Equity,0.155091,-0.172455,0.596940,-0.194819,-0.190088,-0.918115
ABCB UN Equity,0.109143,-0.097693,0.270851,-0.240706,-0.610050,-0.354256
ABG UN Equity,0.053405,0.172590,0.110077,-0.170802,-0.500345,-0.449576


In [ ]:
betas = fdf\
.groupby('DATE', group_keys = False).apply(lambda x: rolling_regression(x, window_size = 24 , dependent_var = 'stock_return', independent_vars = fcols))\
.drop(columns = 'intercept')

In [ ]:
betas

In [ ]:
# this is ignoring a lot of the features, but need this for now to proceed with just model fitting 
# goal is daily data, 


In [61]:
fdf.to_csv('../../bbgfactor/rty_2014_smallfeatures.csv')

### testing dataframe 